In [10]:
import numpy as np


In [1]:
def check_optimality(gk,Hk,tolT): 
#   Revisa que se cumplan las condiciones de optimalidad: Grad=0, Hess positiva semidefinida

#   gk - Vector Gradiente de f en xk
#   Hk - Matriz Hessiana de f en xk
#   tolT - Tolerancia de el gradiente

    return es_pos_def(Hk) and all(abs(gk)<tolT)


In [32]:
def Grad(f, xk, h):
#   Calcula el gradiende de f en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n = xk.size
    g = np.zeros(n)
    for i in range(0, n):
        b = np.copy(xk)
        b[i] += h
        g[i] = (f(b) - f(xk))/(h)
    return g

In [3]:
def Hess(f,xk,h):
#   Calcula el Hessiano en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n=xk.size
    H=np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            ff=np.copy(xk)
            ff[i]+=h
            ff[j]+=h
            
            fb=np.copy(xk)
            fb[i]+=h
            fb[j]-=h
            
            bf=np.copy(xk)
            bf[i]-=h
            bf[j]+=h
            
            bb=np.copy(xk)
            bb[i]-=h
            bb[j]-=h
            
            H[i,j]=(f(ff)-f(fb)-f(bf)+f(bb))/(4*h**2)
    return H
            

In [4]:
def es_pos_def(A):
#   Prueba si la matriz A es positiva semidefinida si A es simétrica

#   A - Matriz a probar

        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
        return False

In [5]:
def PruebaMod_Hess(Hk, Beta ) :
#   Modifica la Hessiana para que sea positiva semidefinida, según el algoritmo 3.3 de Nocedal Ed. 2

#   Hk - Matrizz hessiana a modificar
#   Beta - Valor arbitrario a aumentar

    n=int(np.sqrt(np.size(Hk)))
    tk=0
    v=np.matrix.diagonal(Hk)
    
    if min(v)<0:
        tk=-min(v)+Beta    
    

    while es_pos_def(Hk+np.eye(n)*tk)==False:
           tk= max(2*tk, Beta);
    
    return Hk+np.eye(n)*tk
     

In [6]:
def BactrackSearch(f,xk,pk,gk,alpha0,c,rho):
#   Busca el paso según el algoritmo de 3.1 de Nocedal Ed. 2

#   f - Función a evaluar
#   xk - punto alrededor del que se evalúa
#   pk -Dirección elegida previamente
#   alpha0 - Valor de paso máximo
#   c - peso sobre la derivada direccional
#   rho - factor de disminución del paso

    alpha_k=alpha0
   

    while f(xk+alpha_k*pk)> f(xk)+c*alpha_k*np.dot(gk,pk):
        
        alpha_k=alpha_k*rho
    return alpha_k
    

In [79]:
def Min_LineSearchNewton(f , x0 , tolT , h1 ,h2 , alpha0, c, rho,maxit):  
#   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
#   con una aproximación de la Hessiana adaptada

#   f-Función
#   x0 - Punto inicial
#   tolT -  Tolerancia sobre condición de optimalidad
#   h1 -Diferencias para gradiente
#   h2 -Diferencias para Hessiana
#   c - peso sobre el gradiente para búsqueda lineal Bactracking
#   rho - factor de disminución del paso para búsqueda lineal Bactracking
#   maxit - Máximo número de Iteraciones

    xk=x0
    n=np.size(x0)
    for k in range(0,maxit):
    
    
        gk=Grad(f,xk,h1)                                       
        Hk=Hess(f,xk,h2)         
        
        if check_optimality(gk,Hk,tolT) :
            break
        
        HessMod=PruebaMod_Hess( Hk, 1e-3 ) 

        pk= -np.linalg.solve(HessMod,gk)
        
        
        alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
        xk_1=xk
        xk=xk+alpha_k*pk
        
        
        print(xk)
        print(f(xk))
            
    return [xk,k]

In [80]:
def rosenbrock1_100(v):
    # a=1 b=100
    return ((1-v[0])**2+100*(v[1]-v[0]**2)**2)


In [81]:
def rosenbrock100_1(v):
    # a=100 b=1
    return ((100-v[0])**2+1*(v[1]-v[0]**2)**2)

In [82]:
def rosenbrock50_50(v):
    # a=50 b=50
    return ((50-v[0])**2+50*(v[1]-v[0]**2)**2)


In [83]:
def rosenbrock5_60(v):
    # a=1 b=100
    return ((5-v[0])**2+60*(v[1]-v[0]**2)**2)

In [84]:
Min_LineSearchNewton(rosenbrock1_100 , np.array([4,5]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.8, 0.9,100)

[4. 5.]
12109.0
[3.99948968 9.25758821]
4549.5053474373235
[ 3.9986453  11.86142876]
1712.8118589323274
[ 3.99723947 13.44935409]
648.3497179180218
[ 3.99494688 14.41064489]
248.89608153309754
[ 3.99121401 14.98091704]
98.98321651114534
[ 3.98513945 15.30003985]
42.701631478814065
[ 3.97527657 15.44663615]
21.539235820400897
[ 3.95931966 15.45776431]
13.529520437856918
[ 3.93366465 15.33924274]
10.41473736111379
[ 3.89291988 15.07078848]
9.075201869918642
[ 3.81521649 14.50599683]
8.174245494705843
[ 3.69253563 13.59374621]
7.418448725653136
[ 3.55277451 12.58124557]
6.684439548634605
[ 3.41994556 11.65701455]
6.00833874290342
[ 3.28845571 10.77629815]
5.378727761923136
[3.16011413 9.95021139]
4.796485810873935
[3.03445245 9.17327215]
4.258917204084405
[2.91167899 8.4447349 ]
3.7643403336035277
[2.7918092  7.76254075]
3.310802039502776
[2.67491384 7.12498355]
2.8964227413916097
[2.56105652 6.53030172]
2.5193167351259396
[2.45030522 5.97675237]
2.177604994918151
[2.34273519 5.46262394]


[array([1.00000988, 1.00001924]), 99]

In [85]:
Min_LineSearchNewton(rosenbrock100_1 , np.array([4,5]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.8, 0.9,100)

[4. 5.]
9337.0
[ 5.99148991 26.18178493]
8932.003858980275
[ 8.17607232 56.98729184]
8528.870387405437
[10.36464821 97.57212699]
8131.5937734994195
[ 12.46516461 145.85920827]
7752.999143279224
[ 14.46928973 200.02045067]
7402.7360297555215
[ 16.50211973 263.28530795]
7053.520867907433
[ 18.62605021 338.0675622 ]
6700.258011807721
[ 20.6412146 417.3418408]
6373.818590406264
[ 22.63370551 503.41044129]
6064.294660588118
[ 24.51496243 592.77566531]
5765.357525039481
[ 26.37636382 687.28497009]
5491.464219596739
[ 28.42284538 799.67179867]
5190.305236705114
[ 30.43966401 918.61698719]
4901.940794519666
[  32.42562356 1043.69349637]
4626.011645041041
[  34.52288813 1185.11164854]
4332.385800174555
[  36.4217108  1319.37242841]
4093.587524562583
[  38.32942231 1462.08344908]
3853.1202117058297
[  40.08244146 1598.77774074]
3651.334628911412
[  41.81414064 1740.42592333]
3449.5371869302626
[  43.51094435 1886.98295402]
3229.693403478065
[  45.49337685 2063.0926975 ]
3013.93527056019
[  47.28

[array([  99.99979942, 9999.95988406]), 99]

In [86]:
Min_LineSearchNewton(rosenbrock50_50 , np.array([4,5]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.6, 0.9,10000)

[4. 5.]
8166.0
[ 4.03049239 13.26302742]
2557.764561047093
[ 4.15494694 16.68154299]
2118.707479299791
[ 4.92926949 23.69811686]
2049.345608168345
[ 5.66942087 31.59459858]
1980.200896021119
[ 6.46483447 41.16145277]
1915.321798179827
[ 7.14225906 50.55295523]
1847.315842785891
[ 7.96484046 62.71614744]
1793.057558439701
[ 8.5385842  72.57823028]
1724.4672995227609
[ 9.26027409 85.09702634]
1681.2191036253212
[ 9.87309826 97.10257319]
1617.218109596999
[ 10.63391441 112.39961114]
1572.8443717702635
[ 11.20384594 125.20133128]
1510.4173822518017
[ 11.96693421 142.51358592]
1470.5909297111182
[ 12.5071744  156.13754577]
1409.9712490050465
[ 13.23573578 174.53425996]
1372.764849052082
[ 13.79397006 189.96206218]
1315.729709066102
[ 14.53719221 210.67053951]
1279.3523343875613
[ 15.06656415 226.721106  ]
1224.2719220811643
[ 15.77006048 248.08499739]
1190.2821830998266
[ 16.32200858 266.10330415]
1138.8479894426905
[ 17.01644897 288.97236642]
1105.1530088004413
[ 17.57137706 308.44544004]


[array([  49.99747525, 2499.74753129]), 107]

In [87]:
Min_LineSearchNewton(rosenbrock5_60 , np.array([4,5]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.8, 0.9,100)

[4. 5.]
7261.0
[4.0003118  9.26416532]
2725.3041810878503
[ 4.00079244 11.87858054]
1023.3023772363298
[ 4.00156319 13.48391686]
384.62325842227614
[ 4.00283366 14.47371255]
144.9518469596896
[ 4.00489986 15.09035418]
55.0113334837224
[ 4.0082564  15.48485062]
21.255956159778975
[ 4.01368725 15.75358254]
8.581363318763078
[ 4.02242485 15.96168405]
3.8127882133546604
[ 4.03635579 16.15829838]
2.0038758474803964
[ 4.05823359 16.38677544]
1.2951452006915107
[ 4.09171187 16.69045694]
0.9850450285541313
[ 4.1460305  17.15720254]
0.7921187960531635
[ 4.22965975 17.86614215]
0.6276378833439636
[ 4.32497398 18.68385714]
0.4835056754650716
[ 4.41502648 19.4731104 ]
0.3646556609654345
[ 4.49925129 20.22607424]
0.26847479209461883
[ 4.57745193 20.93798384]
0.19219549863042779
[ 4.6421812  21.53706656]
0.13783364232354717
[ 4.7029719  22.10697069]
0.09545142403695245
[ 4.75815319 22.63072677]
0.06367373806565499
[ 4.80736101 23.10300464]
0.040681274458405456
[ 4.85041363 23.52026476]
0.02471804093

[array([ 4.999967  , 24.99966995]), 41]